<a href="https://colab.research.google.com/github/dondon1895/AI/blob/master/dragon1111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
gpu = GPUs[0]

def printGPU():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  
printGPU()

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
!rm -rf ./logs/

In [ ]:
!python3 --version
!pip3 --version

In [ ]:
!sudo apt update
!sudo apt install python3-dev python3-pip python3-venv

In [ ]:
!python -c "import tensorflow as tf;print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!wget https://yarnpkg.com/latest.tar.gz
!tar zvxf latest.tar.gz

In [ ]:
!wget -qO- https://dl.yarnpkg.com/debian/pubkey.gpg | gpg --import
!wget https://yarnpkg.com/latest.tar.gz.asc
!gpg --verify latest.tar.gz.asc

In [ ]:
!npm install --global yarn

In [ ]:
!yarn --version

In [ ]:
!yarn && yarn watch
!yarn
!yarn train shakespeare \
    --lstmLayerSize 128,128 \
    --epochs 120 \
    --savePath ./my-shakespeare-model
!yarn gen shakespeare ./my-shakespeare-model/model.json \
    --genLength 250 \
    --temperature 0.6

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

text_as_int1 = tf.keras.utils.get_file('alllog.txt','https://raw.githubusercontent.com/dondon1895/work1104/main/alllog.TXT')

In [ ]:
text = open(text_as_int1, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

In [ ]:
print(text[9505:9702])

In [ ]:
n = len(text)
w = len(set(text))
print(f"金瓶梅小說共有 {n} 中文字")
print(f"包含了 {w} 個獨一無二的字")

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import datetime

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:

# 初始化一個以字為單位的 Tokenizer
num_words=4499
tokenizer = tf.keras\
    .preprocessing\
    .text\
    .Tokenizer(
        num_words=num_words,
        char_level=True,
        filters=''
)
    
tokenizer.fit_on_texts(text)
text_as_int = tokenizer\
        .texts_to_sequences([text])[0]

# 隨機選取一個片段文本方便之後做說明
s_idx = 21004
e_idx = 21020
partial_indices = \
    text_as_int[s_idx:e_idx]
partial_texts = [
    tokenizer.index_word[idx] \
    for idx in partial_indices
]

# 渲染結果，可忽略
print("原本的中文字序列：")
print()
print(partial_texts)
print()
print("-" * 20)
print()
print("轉換後的索引序列：")
print()
print(partial_indices)

In [ ]:
text_as_int[:10]

In [ ]:
_type = type(text_as_int)
n = len(text_as_int)
print(f"text_as_int 是一個 {_type}\n")
print(f"小說的序列長度： {n}\n")
print("前 5 索引：", text_as_int[:5])

In [ ]:
print("實際丟給模型的數字序列：")
print(partial_indices[:-1])
print()
print("方便我們理解的文本序列：")
print(partial_texts[:-1])

In [ ]:
# 方便說明，實際上我們會用更大的值來
# 讓模型從更長的序列預測下個中文字
SEQ_LENGTH = 10  # 數字序列長度
BATCH_SIZE = 128 # 幾筆成對輸入/輸出

# text_as_int 是一個 python list
# 我們利用 from_tensor_slices 將其
# 轉變成 TensorFlow 最愛的 Tensor <3
characters = tf\
    .data\
    .Dataset\
    .from_tensor_slices(
        text_as_int)

# 將被以數字序列表示的天龍八部文本
# 拆成多個長度為 SEQ_LENGTH (10) 的序列
# 並將最後長度不滿 SEQ_LENGTH 的序列捨去
sequences = characters\
    .batch(SEQ_LENGTH + 1, 
           drop_remainder=True)

# 天龍八部全文所包含的成對輸入/輸出的數量
steps_per_epoch = \
    len(text_as_int) // SEQ_LENGTH

# 這個函式專門負責把一個序列
# 拆成兩個序列，分別代表輸入與輸出
# （下段有 vis 解釋這在做什麼）
def build_seq_pairs(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# 將每個從文本擷取出來的序列套用上面
# 定義的函式，拆成兩個數字序列
# 作為輸入／輸出序列
# 再將得到的所有數據隨機打亂順序
# 最後再一次拿出 BATCH_SIZE（128）筆數據
# 作為模型一次訓練步驟的所使用的資料
ds = sequences\
    .map(build_seq_pairs)\
    .shuffle(steps_per_epoch)\
    .batch(BATCH_SIZE, 
           drop_remainder=True)

In [ ]:
# print 是用來幫你理解 tf.data.Dataset
# 的內容，實際上存取資料集非常簡單
# 現在先關注下面的 print 結果
for b_inp, b_tar in ds.take(1):
    print("起始句子的 batch：")
    print(b_inp, "\n")
    print("目標句子的 batch：")
    print(b_tar, "\n")
    print("-" * 20, "\n")
    
    print("第一個起始句子的索引序列：")
    first_i = b_inp.numpy()[0]
    print(first_i, "\n")
    print("第一個目標句子的索引序列：")
    first_t = b_tar.numpy()[0]
    print(first_t, "\n")
    print("-" * 20, "\n")
    
    d = tokenizer.index_word
    print("第一個起始句子的文本序列：")
    print([d[i] for i in first_i])
    print()
    print("第一個目標句子的文本序列：")
    print([d[i] for i in first_t])

In [ ]:
for b_inp, b_tar in ds.take(1):
    # 蒙多想去哪就去哪
    # 想怎麼存取 b_iup, b_tar 都可以
    print("b_inp 是個 Tensor：\n")
    print(b_inp)
    print("\nb_tar 也是個 Tensor，")
    print("只是每個數字序列都是"
          "對應的輸入序列往左位"
          "移一格的結果\n")
    print(b_tar)

In [ ]:
# 超參數
EMBEDDING_DIM = 512
RNN_UNITS = 1024

# 使用 keras 建立一個非常簡單的 LSTM 模型
model = tf.keras.Sequential()

# 詞嵌入層
# 將每個索引數字對應到一個高維空間的向量
model.add(
    tf.keras.layers.Embedding(
        input_dim=num_words, 
        output_dim=EMBEDDING_DIM,
        batch_input_shape=[
            BATCH_SIZE, None]
))

# LSTM 層
# 負責將序列數據依序讀入並做處理
model.add(
    tf.keras.layers.LSTM(
    units=RNN_UNITS, 
    return_sequences=True, 
    stateful=True, 
    recurrent_initializer='glorot_uniform'
))

# 全連接層
# 負責 model 每個中文字出現的可能性
model.add(
    tf.keras.layers.Dense(
        num_words))

model.summary()

In [ ]:
# 超參數，決定模型一次要更新的步伐有多大
LEARNING_RATE = 0.001

# 定義模型預測結果跟正確解答之間的差異
# 因為全連接層沒使用 activation func
# from_logits= True 
def loss(y_true, y_pred):
    return tf.keras.losses\
    .sparse_categorical_crossentropy(
        y_true, y_pred, from_logits=True)

# 編譯模型，使用 Adam Optimizer 來最小化
# 剛剛定義的損失函數
model.compile(
    optimizer=tf.keras\
        .optimizers.Adam(
        learning_rate=LEARNING_RATE), 
    loss=loss
)

In [ ]:
EPOCHS = 10 # 決定看幾篇金瓶梅文本
history = model.fit(
    ds, # 前面使用 tf.data 建構的資料集
    epochs=EPOCHS
)

In [ ]:
callbacks = [
    tf.keras.callbacks\
        .TensorBoard("logs"),
    # 你可以加入其他 callbacks 如
    # ModelCheckpoint,
    # EarlyStopping
]

history = model.fit(
    ds,
    epochs=EPOCHS, 
    callbacks=callbacks
)

In [ ]:
pip install -U tensorboard-plugin-profile

In [ ]:
!tensorboard dev upload \
  --logdir logs/fit \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot

In [ ]:

%tensorboard --logdir logs

In [ ]:
# 方便說明，實際上我們會用更大的值來
# 讓模型從更長的序列預測下個中文字
SEQ_LENGTH = 10  # 數字序列長度
BATCH_SIZE = 128 # 幾筆成對輸入/輸出

# text_as_int 是一個 python list
# 我們利用 from_tensor_slices 將其
# 轉變成 TensorFlow 最愛的 Tensor <3
characters = tf\
    .data\
    .Dataset\
    .from_tensor_slices(
        text_as_int)

# 將被以數字序列表示的天龍八部文本
# 拆成多個長度為 SEQ_LENGTH (10) 的序列
# 並將最後長度不滿 SEQ_LENGTH 的序列捨去
sequences = characters\
    .batch(SEQ_LENGTH + 1, 
           drop_remainder=True)

# 天龍八部全文所包含的成對輸入/輸出的數量
steps_per_epoch = \
    len(text_as_int) // SEQ_LENGTH

# 這個函式專門負責把一個序列
# 拆成兩個序列，分別代表輸入與輸出
# （下段有 vis 解釋這在做什麼）
def build_seq_pairs(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# 將每個從文本擷取出來的序列套用上面
# 定義的函式，拆成兩個數字序列
# 作為輸入／輸出序列
# 再將得到的所有數據隨機打亂順序
# 最後再一次拿出 BATCH_SIZE（128）筆數據
# 作為模型一次訓練步驟的所使用的資料
ds = sequences\
    .map(build_seq_pairs)\
    .shuffle(steps_per_epoch)\
    .batch(BATCH_SIZE, 
           drop_remainder=True)

In [ ]:
# 跟訓練時一樣的超參數，
# 只差在 BATCH_SIZE 為 1
EMBEDDING_DIM = 512
RNN_UNITS = 1024
BATCH_SIZE = 1

# 專門用來做生成的模型
infer_model = tf.keras.Sequential()

# 詞嵌入層
infer_model.add(
    tf.keras.layers.Embedding(
        input_dim=num_words, 
        output_dim=EMBEDDING_DIM,
        batch_input_shape=[
            BATCH_SIZE, None]
))

# LSTM 層
infer_model.add(
    tf.keras.layers.LSTM(
    units=RNN_UNITS, 
    return_sequences=True, 
    stateful=True
))

# 全連接層
infer_model.add(
    tf.keras.layers.Dense(
        num_words))
        
infer_model.summary()

In [ ]:
# 超參數，決定模型一次要更新的步伐有多大
LEARNING_RATE = 0.001

# 定義模型預測結果跟正確解答之間的差異
# 因為全連接層沒使用 activation func
# from_logits= True 
def loss(y_true, y_pred):
    return tf.keras.losses\
    .sparse_categorical_crossentropy(
        y_true, y_pred, from_logits=True)

# 編譯模型，使用 Adam Optimizer 來最小化
# 剛剛定義的損失函數
infer_model.compile(
    optimizer=tf.keras\
        .optimizers.Adam(
        learning_rate=LEARNING_RATE), 
    loss=loss
)

In [ ]:
infer_model.load_weights(ckpt_path)
infer_model.build(
    tf.TensorShape([1, None]))

In [ ]:
# 代表「喬」的索引
seed_indices = [234] 

# 增加 batch 維度丟入模型取得預測結果後
# 再度降維，拿掉 batch 維度
input = tf.expand_dims(
    seed_indices, axis=0)
predictions = infer_model(input)
predictions = tf.squeeze(
    predictions, 0)

# 利用生成溫度影響抽樣結果
 predictions \= temperature

# 從 4330 個分類值中做抽樣
# 取得這個時間點模型生成的中文字
sampled_indices = tf.random\
    .categorical(
        predictions, num_samples=1)

In [ ]:
model.save("model.h5")

In [ ]:
tensorflowjs_converter \
    --input_format=keras \
    model.h5 \
    tfjs_model_folder

SyntaxError: ignored

In [ ]:
model = tf.loadLayersModel("url");
const output = model.predict(input);

SyntaxError: ignored